In [74]:
from google.cloud import bigquery
import pandas as pd

# initiating client for bigquery and setting location
client = bigquery.Client(location="europe-west2")

In [72]:
# WHERE construction_age_band invalid, empty string and NO DATA! as null
# remove ENGLAND AND WALES: 
# ENGLAND AND WALES: 2007 ONWARDS

query = ("""
SELECT

    UPPER(lmk_key) AS lmk_key,
    UPPER(guid) AS guid,
    UPPER(current_energy_rating) AS current_energy_rating,
    UPPER(potential_energy_rating) AS potential_energy_rating,
    UPPER(construction_age_band) AS construction_age_band, 

    UPDATE construction_age_band
    SET construction_age_band = NULL
    WHERE construction_age_band = NO DATA!'


FROM ons-ids-analysis-prod.demo_wip_notebook.epc_filtered1


""")

query_job = client.query(
    query,
    # Location must match that of the dataset(s) referenced in the query.
    location="europe-west2",
)  # API request - starts the query

In [11]:
query = ("""
UPDATE ons-ids-analysis-prod.demo_wip_notebook.epc_filtered1

SET construction_age_band = NULL

WHERE construction_age_band like '%NO DATA!%'

""")

query_job = client.query(
    query,
    # Location must match that of the dataset(s) referenced in the query.
    location="europe-west2",
)

#OR construction_age_band = ''
#OR construction_age_band = 'INVALID!'

In [ ]:
query = ("""
SELECT
    UPPER(lmk_key) AS lmk_key,
    UPPER(guid) AS guid,
    UPPER(current_energy_rating) AS current_energy_rating,
    UPPER(potential_energy_rating) AS potential_energy_rating,
    UPPER(construction_age_band) AS construction_age_band,
    CASE
    WHEN construction_age_band IS NOT NULL THEN construction_age_band
    WHEN construction_age_band IN ('NO DATA!') THEN 'NULL'
    ELSE 'no NULL'
    END AS construction_age_band
FROM ons-ids-analysis-prod.demo_wip_notebook.epc_filtered1

""")

query_job = client.query(
    query,
    # Location must match that of the dataset(s) referenced in the query.
    location="europe-west2",
)  # API request - starts the query

In [95]:
query = ("""
SELECT
    CASE
    WHEN construction_age_band IS NOT NULL THEN UPPER(construction_age_band)
    WHEN construction_age_band IN ('NO DATA!') THEN NULL
    ELSE 'no NULL'
    END AS construction_age_band,
    UPPER(lmk_key) AS lmk_key,
    UPPER(guid) AS guid,
    UPPER(current_energy_rating) AS current_energy_rating,
    UPPER(potential_energy_rating) AS potential_energy_rating,
    UPPER(construction_age_band) AS construction_age_band,
FROM ons-ids-analysis-prod.demo_wip_notebook.epc_filtered1

""")

query_job = client.query(
    query,
    # Location must match that of the dataset(s) referenced in the query.
    location="europe-west2",
)  # API request - starts the query

In [96]:
df = query_job.to_dataframe()

In [97]:
df

,construction_age_band,lmk_key,guid,current_energy_rating,potential_energy_rating,construction_age_band_1
0,ENGLAND AND WALES: 1983-1990,366234242132019020812092919068206,0E119516-E71B-4A2E-BBD5-82DA70DEB885,D,A,ENGLAND AND WALES: 1983-1990
1,ENGLAND AND WALES: 1967-1975,1338890055412015071017035092050739,BAF05F2C-D657-46DF-A01E-388D736488EB,D,A,ENGLAND AND WALES: 1967-1975
2,ENGLAND AND WALES: 2003-2006,238847319142020020617141550819568,2D7D73A5-6B73-4F8D-AAE5-9D98B5EFD2B5,D,A,ENGLAND AND WALES: 2003-2006
3,ENGLAND AND WALES: BEFORE 1900,1671442189102018102516403366089538,1E184C87-72CC-4B25-97E2-26D949D5D578,D,A,ENGLAND AND WALES: BEFORE 1900
4,ENGLAND AND WALES: BEFORE 1900,62097539022018040711353104268918,9D64C0F9-8F2E-45EE-92D3-51347C8C3C1D,D,A,ENGLAND AND WALES: BEFORE 1900
...,...,...,...,...,...,...
850361,no NULL,1763401259242019110516391865710758,0B63B354-218A-499D-A689-EE3FD428F764,A,A,None
850362,no NULL,1110014219602014031913465228149418,78AFA655-5BA6-45F3-B665-58286E201E8B,A,A,None
850363,no NULL,1459927049022016070811562925188506,E11B40D7-CF5E-4540-8253-1F3A8C7C4E31,A,A,None
850364,ENGLAND AND WALES: 2007 ONWARDS,1242661589222014112817275990488614,B86E5330-06A4-4129-9F71-C51F15DEAD5A,A,A,ENGLAND AND WALES: 2007 ONWARDS


In [100]:
df[df['construction_age_band'].isnull() == True]

,construction_age_band,lmk_key,guid,current_energy_rating,potential_energy_rating,construction_age_band_1


In [64]:
df[df['guid'].duplicated()==True]

,guid,address,current_energy_rating,potential_energy_rating,construction_age_band


In [ ]:
# checked for value counts
# checked for null values

In [31]:
 # Construct a reference to the wip notebook dataset, within the ons-ids-analysis-prod project
    
PROJECT_ID = 'ons-ids-analysis-prod'
dataset_ref = client.dataset('demo_wip_notebook', project=PROJECT_ID)

In [32]:
# Make an API request to fetch the dataset
dataset = client.get_dataset(dataset_ref)

In [34]:
sql = """
SELECT *
WHERE

FROM ons-ids-analysis-prod.demo_wip_notebook.epc_filtered1
"""

table_ref = dataset.table("epc_upper")
job_config = bigquery.QueryJobConfig(
    destination=table_ref
)

# Start the query, passing in the extra configuration.
query_job = client.query(sql, location="europe-west2", job_config=job_config)

query_job.result()  # Waits for the query to finish
print("Query results loaded to table {}".format(table_ref.path))

Query results loaded to table /projects/ons-ids-analysis-prod/datasets/demo_wip_notebook/tables/epc_upper


In [10]:
# set data types
# ensure energy rating e.g. is a capital letter between A-E
# upper case variables
# strip leading/trailing spaces
# strip non-alphanumeric characters

In [42]:
df

,guid
0,7B4EF54B-BF70-408E-9DC3-F038988FDD5B
1,535B2592-1AB0-4518-9B12-15C4DF55D5D9
2,E06DE197-D667-4BA0-A5A1-9D471CB8318F
3,EF9B99CA-524A-41D1-B853-BD587EE9EC86
4,B09CEC47-7C96-4447-B6DE-E443594CEBD3
...,...
850361,B1C41541-7E7D-4D62-998D-00E49084D3CD
850362,0AE5E4C2-190B-467D-91B2-780420B76F8C
850363,FC4C8C2B-D344-490A-9414-A4E5E0CF62E9
850364,2B74A722-1112-4DCF-A7C4-D6A50BB1DAB2


In [5]:
# we can then pass this to a pandas dataframe
df = query_job.to_dataframe()
'''
However, when we get to actually use the IDS properly, we should do this sparingly.

BigQuery does lots of magic, efficiency-saving stuff in the background that makes doing
your data processing in BigQuery, where possible, a much better proposition than doing so
on a spark or pandas dataframe.

This probably means we're going to have to get really good at SQL.
'''

"\nHowever, when we get to actually use the IDS properly, we should do this sparingly.\n\nBigQuery does lots of magic, efficiency-saving stuff in the background that makes doing\nyour data processing in BigQuery, where possible, a much better proposition than doing so\non a spark or pandas dataframe.\n\nThis probably means we're going to have to get really good at SQL.\n"

In [9]:
df['posttown'].value_counts()

BRISTOL                 12553
MANCHESTER              11829
LEICESTER               11492
NOTTINGHAM              11245
LIVERPOOL               10723
                        ...  
RICKINGHALL INFERIOR        1
CWMGORS                     1
KEWSTOKE                    1
PERTENHALL                  1
Brixham                     1
Name: posttown, Length: 5576, dtype: int64

In [ ]:
# create tiny geography index
GI = pd.DataFrame({'local_authority': [],
                   'geography_entry_id': []
                  })



In [ ]:
# join to geography index
df = pd.merge(GI, on = 'geography_entry_id', how = 'inner')

In [ ]:
# analysis

#